# Setup

## API Keys

In [ ]:
financialmodelingprep_api_key = ""
financialmodelingprep_host= 'https://financialmodelingprep.com/api/v3'

## Libs

In [ ]:
# Installation of necessary libraries
!pip3 install minisom
!pip3 install tslearn
!pip3 install pandas
!pip3 install yfinance
!pip3 install seaborn
!pip3 install plotly
!pip3 install pandas_datareader
!pip3 install fastdtw

# Import necessary libraries
from joblib import Parallel, delayed
from fastdtw import fastdtw
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly as pl
import requests
import datetime
import sys
import os
import math
import time
from scipy.stats import pearsonr
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from plotly.offline import init_notebook_mode , iplot
import plotly.graph_objs as go
from minisom import MiniSom
from tslearn.metrics import dtw_path
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.metrics import dtw
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas_datareader as pdr
import yfinance as yf

# Initialization
sns.set_style("whitegrid")
init_notebook_mode(connected = True)

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from minisom import MiniSom
import yfinance as yf  # To fetch stock data

# Load stock data using yfinance
tickers = ["AAPL", "GOOGL", "MSFT", "AMZN"]
start_date = "2020-01-01"
end_date = "2021-01-01"
data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Normalize data for better training of SOM
normalized_data = (data - data.min()) / (data.max() - data.min())

# Train a SOM
som_dimension = 5
som = MiniSom(som_dimension, som_dimension, normalized_data.shape[1], sigma=0.5, learning_rate=0.5)
som.train_random(normalized_data.values, 1000)

# Get the SOM's representation for each date
winning_coordinates = np.array([som.winner(x) for x in normalized_data.values])

# Prepare data for visualization
mapped_data = pd.DataFrame(winning_coordinates, columns=['x', 'y'])
mapped_data['Date'] = data.index

mapped_data['Date'] = mapped_data['Date'].astype(str)

# Visualize using Plotly
fig = px.scatter(mapped_data, x='x', y='y', animation_frame='Date', range_color=[0, som_dimension-1])
fig.write_html("som_plot.html")


import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=mapped_data['x'], y=mapped_data['y'], mode='markers'))
fig.update_layout(xaxis=dict(range=[mapped_data['x'].min(), mapped_data['x'].max()]),
                  yaxis=dict(range=[mapped_data['y'].min(), mapped_data['y'].max()]))

fig.write_html("som_plot2.html")

fig.show()

# Data

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
#base_data_path  = 'drive/MyDrive/Drive/MSc/Dissertation/StocksData/'

#base_data_path  = '/Users/nenuadrian/Desktop/StocksData/'
base_data_path = '/'

if not os.path.isdir(f"{base_data_path}stock_prices"):
  os.mkdir(f"{base_data_path}stock_prices")

initial_stock_prices = None
dtw_cache = False

## FinancialModelingPrep Profile Data

### Load Companies: NASDAQ & S&P

In [ ]:
def load_companies(path, api):
  companies_path = f"{base_data_path}{path}.csv"
  if not os.path.isfile(companies_path):
    response = requests.get(f'{financialmodelingprep_host}/{api}?apikey={financialmodelingprep_api_key}')
    companies = pd.DataFrame.from_records(response.json())
    companies.to_csv(companies_path)
    print(f"Downloaded {path} companies")
  else:
    print(f"Loading {path} companies from cache")
    companies = pd.read_csv(companies_path)

  print(f"{len(companies)} {path} companies loaded")
  return companies


In [ ]:
sp_companies = load_companies('companies_sp', 'sp500_constituent')
nasdaq_companies = load_companies('companies_nasdaq', 'nasdaq_constituent')

### Company profiles




In [ ]:
initial_symbols = np.concatenate([nasdaq_companies["symbol"].values, sp_companies["symbol"].values])
print(f"Identified {len(initial_symbols)} NASDAQ + S&P symbols")
initial_symbols = list(set(initial_symbols))
print(f"Identified {len(initial_symbols)} unique symbols")

In [ ]:
companies_profiles_path = f"{base_data_path}companies_profiles.csv"
if not os.path.isfile(companies_profiles_path):
  companies_profiles = None
  for index, stock_symbol in enumerate(initial_symbols):
    if companies_profiles is None or companies_profiles.empty or len(companies_profiles.loc[companies_profiles['symbol'] == stock_symbol]) == 0:
      # Make the request to the API
      print(f"Downloaded data for {stock_symbol} {index}/{len(initial_symbols)}")
      response = requests.get(f'{financialmodelingprep_host}/profile/{stock_symbol}?apikey={financialmodelingprep_api_key}')
      if companies_profiles is None:
        companies_profiles = [response.json()[0]]
        companies_profiles = pd.DataFrame.from_records(companies_profiles)
      else:
        row = pd.DataFrame.from_records([response.json()[0]])
        companies_profiles = pd.concat([companies_profiles, row])
  companies_profiles.to_csv(companies_profiles_path)
else:
  companies_profiles = pd.read_csv(companies_profiles_path)

companies_profiles = {
    stock_symbol: companies_profiles.loc[companies_profiles['symbol'] == stock_symbol] for stock_symbol in initial_symbols
}

print(f"{len(companies_profiles)} loaded company profiles")

### Historical Stock Prices

In [ ]:
prices_path = f"{base_data_path}stock_prices/"
start = '2015-01-01'
end = '2023-01-01'
for index, stock_symbol in enumerate(initial_symbols):
  stock_path = f"{prices_path}{stock_symbol}.csv"
  if not os.path.isfile(stock_path):
    print(f"Downloaded data for {stock_symbol} {index}/{len(initial_symbols)}")
    response = requests.get(f'{financialmodelingprep_host}/historical-price-full/{stock_symbol}?apikey={financialmodelingprep_api_key}&from={start}&to={end}')
    data = pd.DataFrame.from_records(response.json()['historical'])
    data.to_csv(stock_path)

## Parse Stock Files

In [ ]:
prices_path = f"{base_data_path}stock_prices/"

if not initial_stock_prices:
  print("Parsing files")
  initial_stock_prices = []
  for stock_symbol in initial_symbols:
    stock_path = f"{prices_path}{stock_symbol}.csv"
    stock_price = pd.read_csv(stock_path)
    stock_price = stock_price.loc[:,["date","close"]]
    stock_price.set_index("date",inplace=True)
    stock_price.sort_index(inplace=True)
    initial_stock_prices.append(stock_price)

print(f"Files parsed from '{prices_path}': {len(initial_stock_prices)}")

## Pre-processing

## Narrowing down the data if configured

In [ ]:
symbols = initial_symbols
stock_prices = initial_stock_prices

print(f"Symbols left: {len(symbols)} - stock prices: {len(stock_prices)}")

### Check for missing data

In [ ]:
def nan_counter(list_of_series):
    nan_polluted_series_counter = 0
    for series in list_of_series:
        if series.isnull().sum().sum() > 0:
            nan_polluted_series_counter+=1
    return nan_polluted_series_counter

print(f"Missing data points: {nan_counter(stock_prices)}")

### Filter out companies with less than 2000 prices

In [ ]:
symbols_without_enough_data = []

for index, symbol in enumerate(symbols):
  if len(stock_prices[index]) < 2000:
    symbols_without_enough_data.append(symbol)

stock_prices = [prices for prices in stock_prices if len(prices) >= 2000]

print(f"Companies without enough data {len(symbols_without_enough_data)}")

for symbol in symbols_without_enough_data:
  symbols.remove(symbol)

companies_profiles = {symbol: companies_profiles[symbol] for symbol in symbols}

print(f"Remaining cleaned symbols {len(symbols)} - stock_prices {len(stock_prices)} - profiles {len(companies_profiles)}")

### Identify a window of time for which all companies have data

In [ ]:
min_start = '2015-01-01'
max_end = '2020-01-01'

for prices in stock_prices:
  if len(prices) > 2000:
    min_start = max(prices.index[0], min_start)
    max_end = min(prices.index[-1], max_end)

print(min_start)
print(max_end)

### Keep only data within the cross-over window

In [ ]:
print(f"Total series before filtering for required timeframe: {len(stock_prices)}")

for index, prices in enumerate(stock_prices):
  stock_prices[index] = prices.filter(items = [date for date in prices.index if date >= min_start and date <= max_end], axis=0)

print(f"Total series after filtering for required timeframe: {len(stock_prices)}")

### Normalize

In [ ]:
stock_prices_normal = []
scaler = MinMaxScaler()
for prices in stock_prices:
  df = prices.copy(deep = True)
  df = MinMaxScaler().fit_transform(df)
  df = df.reshape(len(df))
  stock_prices_normal.append(df)

print(f"Normalised stocks: {len(stock_prices_normal)}")

# Explore

## Industries, sectors and countries

In [ ]:
industries = [companies_profiles[symbol]["industry"].values[0] for symbol in companies_profiles]
sectors = [companies_profiles[symbol]["sector"].values[0] for symbol in companies_profiles]
countries = [companies_profiles[symbol]["country"].values[0] for symbol in companies_profiles]

In [ ]:
sectorsDf = pd.DataFrame(sectors,columns=["Sector"]).groupby("Sector")["Sector"].count().sort_values(ascending=False).reset_index(name='count')
sectorsDf

In [ ]:
industriesDf = pd.DataFrame(industries,columns=["Industry"]).groupby("Industry")["Industry"].count().sort_values(ascending=False).reset_index(name='count')

industriesDf


In [ ]:
sectorsGroup = pd.DataFrame(sectors,columns=["sector"]).groupby("sector")["sector"].count().sort_values(ascending=False).reset_index(name='count')
plt.figure(figsize=(7,7))
plt.rcParams['font.size'] = 14

plt.pie(sectorsGroup["count"], labels = sectorsGroup["sector"], shadow = True)
plt.show()
sectorsGroup

In [ ]:
countries = pd.DataFrame(countries,columns=["country"]).groupby("country")["country"].count().sort_values(ascending=False).reset_index(name='count')
countries

## Stocks

In [ ]:
plots = stock_prices[:30]

fig, axs = plt.subplots(math.floor(len(plots) / 3),3,figsize=(25,45))
for i in range(math.floor(len(plots) / 3) + 1):
  for j in range(3):
      if i*3+j+1>len(plots): # pass the others that we can't fill
          continue
      axs[i, j].plot(plots[i*3+j].values)
      axs[i, j].set_title(symbols[i*3+j])
plt.show()

In [ ]:

fig = plt.figure(figsize=(25,5))
for stock in stock_prices:
  plt.plot(stock)
plt.show()

In [ ]:
fig = plt.figure(figsize=(25,5))
for stock in stock_prices_normal:
  plt.plot(stock)
plt.show()

## DTW Paths

In [ ]:
stock1 = stock_prices_normal[0]
stock2 = stock_prices_normal[30]

path, sim = dtw_path(stock1, stock2)

plt.figure(figsize=(8, 8))

# Plot stock1
plt.subplot(2, 2, 1)
plt.plot(stock1, label='Stock 1')
plt.legend()
plt.grid()

# Plot stock2
plt.subplot(2, 2, 4)
plt.plot(stock2, label='Stock 2')
plt.legend()
plt.grid()

# Plot DTW path
plt.subplot(2, 2, 2)
for (i, j) in path:
    plt.plot([i, j], [stock1[i], stock2[j]], color='gray', linewidth=0.5)

plt.grid()

plt.show()

In [ ]:
# Create a matrix with zeros
mat = np.zeros((len(stock1), len(stock2)))

# Update the matrix cells to one, corresponding to the DTW path
for i, j in path:
    mat[i, j] = 1

# Plotting
fig, ax = plt.subplots(figsize=(8, 8))

# Show the matrix
cax = ax.matshow(mat.T, origin='lower', cmap='gray_r')
fig.colorbar(cax)

plt.plot([j for (i, j) in path], [i for (i, j) in path], "w-", linewidth=2.)  # DTW path

plt.title('DTW Warping Path')
plt.xlabel('Stock 1')
plt.ylabel('Stock 2')

plt.tight_layout()
plt.show()

In [ ]:
stock1 = np.random.rand(100)
stock2 = np.random.rand(100) + 1   # just to make the two series visibly different
path, sim = dtw_path(stock1, stock2)

# Plot the two time series
plt.figure(figsize=(12, 6))
plt.plot(stock1, 'r-', label='stock1')
plt.plot(stock2, 'g-', label='stock2')

# Draw lines between matched points according to the warping path
for (i, j) in path:
    plt.plot([i, j], [stock1[i], stock2[j]], 'b-', linewidth=0.1)

plt.legend()
plt.title('Dynamic Time Warping')
plt.show()

In [ ]:
# Assume these are your time series

# Plotting
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)

plt.plot(stock1, 'r', label='stock1')
plt.plot(stock2, 'g', label='stock2')
plt.legend()

# Creating the connections for the Euclidean distance
for i in range(len(stock1)):
    plt.plot([i, i], [stock1[i], stock2[i]], 'b')

plt.title('Euclidean Distance')
plt.show()

# Analysis

## Correlation Heatmap

In [ ]:
# Assuming each stock has the same dates
close_prices = pd.concat([stock_prices[i]['close'] for i in range(len(stock_prices))], axis=1)
close_prices.columns = ['Stock {}'.format(i) for i in range(len(stock_prices))]
correlation_matrix = close_prices.corr()
correlation_matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have `stock_prices` as a list of dataframes
close_prices = pd.concat([stock_prices[i]['close'] for i in range(100)], axis=1)
close_prices.columns = ['Stock {}'.format(i + 1) for i in range(100)]
correlation_matrix = close_prices.corr()

# Set up the matplotlib figure
plt.figure(figsize=(20, 20))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation_matrix, cmap="coolwarm", center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .75})

plt.show()

In [ ]:
#plt.figure(figsize=(14,7))
#sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
#plt.title('Correlation Heatmap of Closing Prices for All Stocks')
#plt.show()

## Average vs DBA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tslearn.barycenters import dtw_barycenter_averaging

# Generate synthetic stock data with different scales
np.random.seed(0)  # for reproducibility
time_points = 100
base = np.linspace(100, 150, time_points)

# First stock: Small fluctuations around the base
stock1 = base + np.random.randn(time_points) * 2

# Second stock: Sinusoidal fluctuations of larger magnitude around the base
stock2 = base + np.random.randn(time_points) * 2 + np.sin(np.linspace(0, 4 * np.pi, time_points)) * 20

# Third stock: Linear trend that is drastically different from the base
stock3 = base + np.linspace(-50, 50, time_points) + np.random.randn(time_points) * 2

stocks = np.array([stock1, stock2, stock3])

# Compute simple average
simple_avg = np.mean(stocks, axis=0)

# Compute DTW Barycenter Averaging (DBA)
dba_avg = dtw_barycenter_averaging(stocks)

# Visualization using matplotlib.pyplot
plt.figure(figsize=(10, 6))

# Plot individual stocks
colors = ['blue', 'orange', 'purple']
for i, stock in enumerate(stocks):
    plt.plot(stock, label=f'Stock {i+1}', color=colors[i], linewidth=1.5)

# Plot simple average
plt.plot(simple_avg, label='Simple Average', color='red', linewidth=2, linestyle='dotted')

# Plot DBA
plt.plot(dba_avg.ravel(), label='DBA Average', color='green', linewidth=2, linestyle='dashed')

plt.legend()
plt.title("Stock Prices: Simple Average vs. DBA")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

# Visualization using matplotlib.pyplot
plt.figure(figsize=(10, 6))

# Plot individual stocks
colors = ['blue', 'orange', 'purple']
for i, stock in enumerate(stocks):
    plt.plot(stock, label=f'Stock {i+1}', color=colors[i], linewidth=1.5)

# Plot simple average
plt.plot(simple_avg, label='Simple Average', color='red', linewidth=2, linestyle='dotted')


plt.legend()
plt.title("Simple Average of Stock prices")
plt.xlabel("Time")
plt.ylabel("Stock Price")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate synthetic data
np.random.seed(0)
cluster1 = np.random.randn(50, 2) + [2, 2]
cluster2 = np.random.randn(50, 2) + [7, 7]
cluster3 = np.random.randn(50, 2) + [2, 7]

data = np.vstack([cluster1, cluster2, cluster3])

# Compute centroids
centroid1 = np.mean(cluster1, axis=0)
centroid2 = np.mean(cluster2, axis=0)
centroid3 = np.mean(cluster3, axis=0)

global_centroid = np.mean(data, axis=0)

# Plot clusters
plt.scatter(cluster1[:, 0], cluster1[:, 1], color='blue', s=10, label='Cluster 1')
plt.scatter(cluster2[:, 0], cluster2[:, 1], color='orange', s=10, label='Cluster 2')
plt.scatter(cluster3[:, 0], cluster3[:, 1], color='green', s=10, label='Cluster 3')

# Plot centroids
plt.scatter([centroid1[0], centroid2[0], centroid3[0]],
            [centroid1[1], centroid2[1], centroid3[1]],
            color='black', s=50, marker='x', label='Centroids')

# Plot global centroid
plt.scatter(global_centroid[0], global_centroid[1], color='red', s=50, marker='x', label='Global Centroid')

# Draw lines to global centroid
for centroid in [centroid1, centroid2, centroid3]:
    plt.plot([centroid[0], global_centroid[0]], [centroid[1], global_centroid[1]], 'r--')

plt.title("Calinski-Harabasz Index")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid(True)
plt.show()


## Utils

In [ ]:
def draw_with_average(labels, k):
  plot_count = math.ceil(math.sqrt(k))
  fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
  row_i=0
  column_j=0
  # For each label there is,
  # plots every series with that label
  for label in set(labels):
    cluster = []
    for i in range(len(labels)):
      if(labels[i]==label):
          axs[row_i, column_j].plot(stock_prices_normal[i],c="gray",alpha=0.4)
          cluster.append(stock_prices_normal[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(row_i*plot_count+column_j + 1))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0

  plt.show()

def compute_dtw_barycenter_averaging(cluster, data):
  return [cluster, dtw_barycenter_averaging(np.vstack(data))]

def draw_with_bary(labels, k):
  plot_count = math.ceil(math.sqrt(k))

  fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
  row_i=0
  column_j=0
  for label in set(clusters_kmeans_dtw):
      cluster = []
      for i in range(len(clusters_kmeans_dtw)):
              if(clusters_kmeans_dtw[i]==label):
                  axs[row_i, column_j].plot(stock_prices_normal[i],c="gray",alpha=0.4)
                  cluster.append(stock_prices_normal[i])
      if len(cluster) > 0:
          axs[row_i, column_j].plot(dtw_barycenter_averaging(np.vstack(cluster)),c="red")
      axs[row_i, column_j].set_title("Cluster "+str(row_i*plot_count+column_j + 1))
      column_j+=1
      if column_j%plot_count == 0:
          row_i+=1
          column_j=0


  plt.show()

## DTW Flat Clusters From Hierarchical Clustering

### Run - PARALLEL

In [ ]:
%%time
from scipy.spatial.distance import squareform


def compute_dtw(i, j):
  return [[i, j], dtw(stock_prices_normal[i], stock_prices_normal[j])]

dtw_matrix_path = f"{base_data_path}dtw_matrix.json"


to_process = []
n = len(stock_prices_normal)  # Number of stocks
for i in range(n):
  for j in range(n):
    if i != j:
      to_process.append([i, j])

results = Parallel(n_jobs=5)(delayed(compute_dtw)(x[0], x[1]) for x in to_process)

dtw_matrix = np.zeros((n, n))
for result in results:
  dtw_matrix[result[0][0], result[0][1]] = result[1]



In [ ]:
import json
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

dtw_matrix_path = f"{base_data_path}dtw_matrix.json"

# Serializing json
json_object = json.dumps(dtw_matrix, cls=NumpyArrayEncoder)

# Writing to sample.json
with open(dtw_matrix_path, "w") as outfile:
    outfile.write(json_object)

In [ ]:
for method in ['average', 'ward']:
  # Perform hierarchical clustering
  condensed_dist_matrix = squareform(dtw_matrix)
  Z = linkage(condensed_dist_matrix, method=method)

  # Create a dendrogram
  plt.figure(figsize=(10, 10))
  dendrogram(Z)
  plt.title(f'Hierarchical DTW Clustering Dendrogram {method}')
  plt.xlabel('Stock')
  plt.ylabel('Distance')
  plt.show()

  hierarchical_k = 23
  hierarchical_labels = fcluster(Z, hierarchical_k, criterion='maxclust')
  draw_with_average(hierarchical_labels, hierarchical_k)

  print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(dtw_matrix, hierarchical_labels, metric="precomputed"))

  print("Calinski-Harabasz Index: %0.3f"
        % metrics.calinski_harabasz_score(stock_prices_normal, hierarchical_labels))

  print("Davies-Bouldin Index: %0.3f"
        % metrics.davies_bouldin_score(stock_prices_normal, hierarchical_labels))


  cluster_map = []
  for idx in range(len(hierarchical_labels)):
      winner_node = hierarchical_labels[idx]
      industry = companies_profiles[symbols[idx]]["industry"].values[0]
      cluster_map.append((winner_node , industry))


  pivot_df = pd.DataFrame(cluster_map,columns=["Cluster", "Industry"]).sort_values(by="Cluster")

  # Count unique industries per cluster
  unique_industries_per_cluster = pivot_df.groupby("Cluster")["Industry"].nunique().reset_index().sort_values(by="Cluster")

  # Plotting the data using matplotlib.pyplot
  plt.figure(figsize=(25,5))
  plt.bar(unique_industries_per_cluster["Cluster"], unique_industries_per_cluster["Industry"],)
  plt.xlabel("Cluster")
  plt.ylabel("Number of Unique Industries")
  plt.title("Number of Unique Industries per Cluster")
  plt.xticks(unique_industries_per_cluster["Cluster"].tolist())
  plt.grid(axis='y', linestyle='--', alpha=0.7)
  plt.tight_layout()
  plt.show()


  cluster_c = [len(hierarchical_labels[hierarchical_labels==i]) for i in range(1, 24)]
  cluster_n = [f"C{i}" for i in range(1, 24)]
  plt.figure(figsize=(25,5))
  plt.title("Cluster Distribution of stocks for DTW Kernel")
  plt.bar(cluster_n,cluster_c)
  plt.show()


### Evaluate

In [ ]:
for cluster in [5]:
  plt.figure(figsize=(25,5))
  plt.title(f"Cluster {cluster}")
  for index, clusterNo in enumerate(clusters):
    if clusterNo == cluster:
      plt.plot(stock_prices_normal[index])
  plt.show()

## MiniSom


### Run

In [ ]:
%%time
def print_stats_som(mini_som_clusters):
  print("Silhouette Coefficient: %0.3f"
        % metrics.silhouette_score(stock_prices_normal, mini_som_clusters))

  print("Calinski-Harabasz Index: %0.3f"
        % metrics.calinski_harabasz_score(stock_prices_normal, mini_som_clusters))

  print("Davies-Bouldin Index: %0.3f"
        % metrics.davies_bouldin_score(stock_prices_normal, mini_som_clusters))

  return [metrics.silhouette_score(stock_prices_normal, mini_som_clusters),
          metrics.calinski_harabasz_score(stock_prices_normal, mini_som_clusters),
          metrics.davies_bouldin_score(stock_prices_normal, mini_som_clusters)]

for sigma in list(np.arange(0.1, 1, .1)):
  print(f"Sigma: {sigma}")
  som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(stock_prices_normal))))

  som = MiniSom(som_x, som_y, len(stock_prices_normal[0]), sigma=sigma, learning_rate = 0.3, random_seed=1)

  som.random_weights_init(stock_prices_normal)
  som.train(stock_prices_normal, 50000)

  # Calculate the quantization error
  q_error = som.quantization_error(stock_prices_normal)
  print("Quantization error:", q_error)

  # Calculate the topographic error
  t_error = som.topographic_error(stock_prices_normal)
  print("Topographic error:", t_error)

  # Calculate the U-matrix
  u_matrix = som.distance_map()
  print("U-matrix:", u_matrix)

  win_map = som.win_map(stock_prices_normal)

  mini_som_clusters = []
  for idx in range(len(stock_prices_normal)):
      winner_node = som.winner(stock_prices_normal[idx])
      mini_som_clusters.append(winner_node[0]*som_y+winner_node[1]+1)

  print_stats_som(mini_som_clusters)


We then calculate the quantization error of the model using the quantization_error method and print it to the console. The quantization error measures how well the model preserves the distances between the input data points.

We also calculate the topographic error of the model using the topographic_error method and print it to the console. The topographic error measures how well the model preserves the topology of the input data, i.e. how well it preserves the relationships between nearby data points.

Finally, we calculate the U-matrix of the model using the distance_map method and print it to the console. The U-matrix is a 2D array that visualizes the distances between adjacent nodes on the grid. It can be used to identify clusters of similar data points.

### Evaluate

In [ ]:
som_results_stats = [print_stats_som(result) for result in som_results]

scaler = MinMaxScaler()
df = MinMaxScaler().fit_transform(pd.DataFrame([result[0] for result in som_results_stats]))
df1 = df.reshape(len(df))

df = MinMaxScaler().fit_transform(pd.DataFrame([result[1] for result in som_results_stats]))
df2 = df.reshape(len(df))

df = MinMaxScaler().fit_transform(pd.DataFrame([result[2] for result in som_results_stats]))
df3 = df.reshape(len(df))

plt.figure(figsize=(12, 6))
plt.plot(range(3, 30), df1, label='Silhouette by Coefficient')
plt.plot(range(3, 30), df2, label='Calinski-Harabasz Index')
plt.plot(range(3, 30), df3, label='Davies-Bouldin Index')
plt.legend()
plt.show()

### Explore

#### Clusters

In [ ]:
for cluster in [1]:
  plt.figure(figsize=(25,5))
  plt.title(f"Cluster {cluster}")
  for index, clusterNo in enumerate(mini_som_clusters):
    if clusterNo == cluster:
      plt.plot(stock_prices_normal[index])
  plt.show()

#### Averaging Centre

In [ ]:
%%time

# Little handy function to plot series
def plot_som_series_averaged_center(som_x, som_y, win_map):
    fig, axs = plt.subplots(som_x,som_y,figsize=(25,25))
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            if cluster in win_map.keys():
                for series in win_map[cluster]:
                    axs[cluster].plot(series,c="gray",alpha=0.5)
                axs[cluster].plot(np.average(np.vstack(win_map[cluster]),axis=0),c="red")
            cluster_number = x*som_y+y+1
            axs[cluster].set_title(f"Cluster {cluster_number}")

    plt.show()

# Returns the mapping of the winner nodes and inputs
print(f"Clusters: {len(win_map)}")
plot_som_series_averaged_center(som_x, som_y, win_map)

#### DTW Barycenter Averaging (DBA) - PARALLEL PROCESSING

In [ ]:
%%time

to_process = []
for x in range(som_x):
  for y in range(som_y):
    cluster = (x,y)
    if cluster in win_map.keys():
      to_process.append([cluster, win_map[cluster]])

numbers = Parallel(n_jobs=5)(delayed(compute_dtw_barycenter_averaging)(x[0], x[1]) for x in to_process)

In [ ]:
fig, axs = plt.subplots(som_x,som_y,figsize=(25,25))
for data in numbers:
  cluster = data[0]
  for series in win_map[cluster]:
    axs[cluster].plot(series,c="gray",alpha=0.5)
  axs[cluster].plot(data[1],c="red")
  cluster_number = data[0][0]*som_y+data[0][1]+1
  axs[cluster].set_title(f"Cluster {cluster_number}")

plt.show()

print(f"Clusters: {len(win_map)}")

In [ ]:
cluster_c = []
cluster_n = []
for x in range(som_x):
    for y in range(som_y):
        cluster = (x,y)
        if cluster in win_map.keys():
            cluster_c.append(len(win_map[cluster]))
        else:
            cluster_c.append(0)
        cluster_number = x*som_y+y+1
        cluster_n.append(f"{cluster_number}")

plt.figure(figsize=(25,5))
plt.title("Cluster Distribution for SOM")
plt.bar(cluster_n,cluster_c)
plt.show()

In [ ]:
cluster_map = []
for idx in range(len(stock_prices_normal)):
    winner_node = som.winner(stock_prices_normal[idx])
    industry = companies_profiles[symbols[idx]]["industry"].values[0]
    cluster_map.append((symbols[idx], f"Cluster {winner_node[0]*som_y+winner_node[1]+1}", industry))

pd.DataFrame(cluster_map,columns=["Series","Cluster", "Industry"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
cluster_map = []
for idx in range(len(stock_prices_normal)):
    winner_node = som.winner(stock_prices_normal[idx])
    industry = companies_profiles[symbols[idx]]["industry"].values[0]
    cluster_map.append((winner_node[0]*som_y+winner_node[1]+1, industry))

pivot_df = pd.DataFrame(cluster_map,columns=["Cluster", "Industry"]).sort_values(by="Cluster")

# Count unique industries per cluster
unique_industries_per_cluster = pivot_df.groupby("Cluster")["Industry"].nunique().reset_index().sort_values(by="Cluster")

# Plotting the data using matplotlib.pyplot
plt.figure(figsize=(25,5))
plt.bar(unique_industries_per_cluster["Cluster"], unique_industries_per_cluster["Industry"],)
plt.xlabel("Cluster")
plt.ylabel("Number of Unique Industries")
plt.title("Number of Unique Industries per Cluster")
plt.xticks(unique_industries_per_cluster["Cluster"].tolist())
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

#### KMeans

In [ ]:
winning_neurons = np.array([som.winner(x) for x in stock_prices_normal])

kmeans_som = KMeans(n_clusters=23)
clusters = kmeans_som.fit_predict(winning_neurons)


In [ ]:
# Evaluate the performance of the model
inertia = kmeans_som.inertia_
print("Inertia:", inertia)

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(stock_prices_normal, kmeans_som.labels_))

print("Calinski-Harabasz Index: %0.3f"
      % metrics.calinski_harabasz_score(stock_prices_normal, kmeans_som.labels_))

print("Davies-Bouldin Index: %0.3f"
      % metrics.davies_bouldin_score(stock_prices_normal, kmeans_som.labels_))

## TimeSeriesKMeans Euclidian

In [ ]:
def print_kmeans_stats(km):
  inertia = km.inertia_
  print("Inertia:", inertia)

  print("Silhouette Coefficient: %0.3f"
        % metrics.silhouette_score(stock_prices_normal, km.labels_))

  print("Calinski-Harabasz Index: %0.3f"
        % metrics.calinski_harabasz_score(stock_prices_normal, km.labels_))

  print("Davies-Bouldin Index: %0.3f"
        % metrics.davies_bouldin_score(stock_prices_normal, km.labels_))

  return [inertia,  metrics.silhouette_score(stock_prices_normal, km.labels_),
          metrics.calinski_harabasz_score(stock_prices_normal, km.labels_),
          metrics.davies_bouldin_score(stock_prices_normal, km.labels_)]

### Run

In [ ]:
%%time
# Apply TimeSeriesKMeans clustering with Euclidean metric
kmeans_cluster_count = math.ceil(math.sqrt(len(stock_prices_normal)))
km_euclidean = TimeSeriesKMeans(n_clusters=kmeans_cluster_count, metric="euclidean", max_iter=1000, n_init=2, n_jobs=-1, random_state=1)
clusters_euclidean = km_euclidean.fit_predict(stock_prices_normal)

### Explore

In [ ]:
cluster_map = []
for idx in range(len(clusters_euclidean)):
    winner_node = clusters_euclidean[idx]
    industry = companies_profiles[symbols[idx]]["industry"].values[0]
    cluster_map.append((winner_node + 1, industry))


pivot_df = pd.DataFrame(cluster_map,columns=["Cluster", "Industry"]).sort_values(by="Cluster")

# Count unique industries per cluster
unique_industries_per_cluster = pivot_df.groupby("Cluster")["Industry"].nunique().reset_index().sort_values(by="Cluster")

# Plotting the data using matplotlib.pyplot
plt.figure(figsize=(25,5))
plt.bar(unique_industries_per_cluster["Cluster"], unique_industries_per_cluster["Industry"],)
plt.xlabel("Cluster")
plt.ylabel("Number of Unique Industries")
plt.title("Number of Unique Industries per Cluster")
plt.xticks(unique_industries_per_cluster["Cluster"].tolist())
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
plot_count = math.ceil(math.sqrt(kmeans_cluster_count))

fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
row_i=0
column_j=0
# For each label there is,
# plots every series with that label
for label in set(clusters_euclidean):
    cluster = []
    for i in range(len(clusters_euclidean)):
            if(clusters_euclidean[i]==label):
                axs[row_i, column_j].plot(stock_prices_normal[i],c="gray",alpha=0.4)
                cluster.append(stock_prices_normal[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(row_i*plot_count+column_j + 1))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0

plt.show()

### Evaluate

In [ ]:
for cluster in [7]:
  plt.figure(figsize=(25,5))
  plt.title(f"Cluster {cluster + 1}")
  for index, clusterNo in enumerate(clusters_euclidean):
    if clusterNo == cluster:
      plt.plot(stock_prices_normal[index])
  plt.show()

In [ ]:
print_kmeans_stats(km_euclidean)

In [ ]:
results = []
for kmeans_cluster_count in range(3, 30):
  km_euclidean = TimeSeriesKMeans(n_clusters=kmeans_cluster_count, metric="euclidean", max_iter=10, n_init=2, n_jobs=-1, random_state=1)
  clusters_euclidean = km_euclidean.fit_predict(stock_prices_normal)
  print(kmeans_cluster_count)
  results.append(print_kmeans_stats(km_euclidean))

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot([result[1] for result in results])
plt.title('Silhouette by Coefficient')
plt.show()

plt.figure(figsize=(12, 6))
plt.plot([result[2] for result in results])
plt.title('Calinski-Harabasz Index')
plt.show()

plt.figure(figsize=(12, 6))
plt.plot([result[3] for result in results])
plt.title('Davies-Bouldin Index')
plt.show()


In [ ]:
scaler = MinMaxScaler()
df = MinMaxScaler().fit_transform(pd.DataFrame([result[1] for result in results]))
df1 = df.reshape(len(df))

df = MinMaxScaler().fit_transform(pd.DataFrame([result[2] for result in results]))
df2 = df.reshape(len(df))

df = MinMaxScaler().fit_transform(pd.DataFrame([result[3] for result in results]))
df3 = df.reshape(len(df))

plt.figure(figsize=(12, 6))
plt.plot(range(3, 30), df1, label='Silhouette by Coefficient')
plt.plot(range(3, 30), df2, label='Calinski-Harabasz Index')
plt.plot(range(3, 30), df3, label='Davies-Bouldin Index')
plt.legend()
plt.show()


In [ ]:
cluster_c = [len(km_euclidean.labels_[km_euclidean.labels_==i]) for i in range(23)]
cluster_n = [f"C{i + 1}" for i in range(23)]
plt.figure(figsize=(25,5))
plt.title("Cluster Distribution of stocks for KMeans Euclidean")
plt.bar(cluster_n,cluster_c)
plt.show()

## TimeSeriesKMeans DTW

### Run

In [ ]:
%%time

kmeans_dtw_cluster_count = math.ceil(math.sqrt(len(stock_prices_normal)))
# A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

print(f"{kmeans_dtw_cluster_count} clusters")

km_dtw = TimeSeriesKMeans(n_clusters=kmeans_dtw_cluster_count, max_iter_barycenter=1000, metric="dtw",n_init=10,  random_state=1, n_jobs=5)

# elbow method - to check for sweet spot number of clusters: play with init https://tslearn.readthedocs.io/en/stable/gen_modules/clustering/tslearn.clustering.TimeSeriesKMeans.html
km_dtw.fit(stock_prices_normal)

clusters_kmeans_dtw = km_dtw.predict(stock_prices_normal)




### Evaluate

In [ ]:
# Evaluate the performance of the model
inertia = km_dtw.inertia_
print("Inertia:", inertia)

print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(stock_prices_normal, km_dtw.labels_))

print("Calinski-Harabasz Index: %0.3f"
      % metrics.calinski_harabasz_score(stock_prices_normal, km_dtw.labels_))

print("Davies-Bouldin Index: %0.3f"
      % metrics.davies_bouldin_score(stock_prices_normal, km_dtw.labels_))



### Explore

#### Clusters

##### Average

In [ ]:
plot_count = math.ceil(math.sqrt(kmeans_dtw_cluster_count))

fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
row_i=0
column_j=0
# For each label there is,
# plots every series with that label
for label in set(clusters_kmeans_dtw):
    cluster = []
    for i in range(len(clusters_kmeans_dtw)):
            if(clusters_kmeans_dtw[i]==label):
                axs[row_i, column_j].plot(stock_prices_normal[i],c="gray",alpha=0.4)
                cluster.append(stock_prices_normal[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(row_i*plot_count+column_j + 1))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0

plt.show()

##### DBA

In [ ]:
draw_with_bary(clusters_kmeans_dtw, kmeans_dtw_cluster_count)

In [ ]:
for cluster in range(len(km_dtw.labels_)):
  plt.figure(figsize=(25,5))
  plt.title(f"Cluster {cluster + 1}")
  for index, clusterNo in enumerate(km_dtw.labels_):
    if clusterNo == cluster:
      plt.plot(stock_prices_normal[index])
  plt.show()

#### Outliars

In [ ]:
for index in range(len(stock_prices_normal)):
  if km_dtw.labels_[index] == 14:
    plt.figure(figsize=(25,5))
    plt.title(f"Cluster 15: {symbols[index]}")
    plt.plot(stock_prices_normal[index])
    plt.show()

plt.figure(figsize=(25,5))
names = []
for index in range(len(stock_prices_normal)):
  if km_dtw.labels_[index] == 19:
    plt.plot(stock_prices_normal[index])
    names.append(symbols[index])

plt.title(f"{', '.join(names)}")
plt.legend(names)
plt.show()




#### Distributions

In [ ]:
cluster_c = [len(km_dtw.labels_[km_dtw.labels_==i]) for i in range(kmeans_dtw_cluster_count)]
cluster_n = [f"C{i + 1}" for i in range(kmeans_dtw_cluster_count)]
plt.figure(figsize=(25,5))
plt.title("Cluster Distribution of stocks for KMeans DTW")
plt.bar(cluster_n,cluster_c)
plt.show()

In [ ]:
cluster_map = []
for idx in range(len(km_dtw.labels_)):
    winner_node = km_dtw.labels_[idx]
    industry = companies_profiles[symbols[idx]]["industry"].values[0]
    cluster_map.append((winner_node + 1, industry))


pivot_df = pd.DataFrame(cluster_map,columns=["Cluster", "Industry"]).sort_values(by="Cluster")

# Count unique industries per cluster
unique_industries_per_cluster = pivot_df.groupby("Cluster")["Industry"].nunique().reset_index().sort_values(by="Cluster")

# Plotting the data using matplotlib.pyplot
plt.figure(figsize=(25,5))
plt.bar(unique_industries_per_cluster["Cluster"], unique_industries_per_cluster["Industry"],)
plt.xlabel("Cluster")
plt.ylabel("Number of Unique Industries")
plt.title("Number of Unique Industries per Cluster")
plt.xticks(unique_industries_per_cluster["Cluster"].tolist())
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
data = pd.DataFrame(zip(km_dtw.labels_, [companies_profiles[d]["industry"].values[0] for d in symbols], symbols), columns=["Cluster", "Industry", "Symbol"])
grouped = data.groupby("Cluster")["Industry"]

pd.DataFrame(grouped.apply(list))

## Principal component analysis (PCA)

### Run

In [ ]:
%%time
pca = PCA()
mySeries_transformed = pca.fit_transform(stock_prices_normal)

# Plot cumulative explained variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)')
plt.title('Explained Variance')
plt.show()

In [ ]:
pca = PCA(n_components=2)

mySeries_transformed = pca.fit_transform(stock_prices_normal)

# The explained variance ratio of each component
print("Explained variance ratio:", pca.explained_variance_ratio_)

# The cumulative sum of the explained variance ratio
print("Cumulative sum of explained variance ratio:", np.cumsum(pca.explained_variance_ratio_))

### Explore

In [ ]:
plt.figure(figsize=(25,10))
plt.scatter(mySeries_transformed[:,0], mySeries_transformed[:,1], s=100)
plt.show()

In [ ]:
plt.figure(figsize=(25,10))

plt.scatter(mySeries_transformed[:, 0], mySeries_transformed[:, 1], s=300)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

plt.title("TimeSeriesKMeans Clustering")
plt.show()

### KMeans

#### Run

In [ ]:
kmeans_cluster_count = math.ceil(math.sqrt(len(stock_prices_normal)))
kmeans = TimeSeriesKMeans(n_clusters=kmeans_cluster_count, metric="dtw",random_state=1)
labels = kmeans.fit_predict(mySeries_transformed)
print(f"{kmeans_cluster_count} clusters")

In [ ]:
cluster_map = []
for idx in range(len(labels)):
    winner_node = labels[idx]
    industry = companies_profiles[symbols[idx]]["industry"].values[0]
    cluster_map.append((winner_node + 1, industry))


pivot_df = pd.DataFrame(cluster_map,columns=["Cluster", "Industry"]).sort_values(by="Cluster")

# Count unique industries per cluster
unique_industries_per_cluster = pivot_df.groupby("Cluster")["Industry"].nunique().reset_index().sort_values(by="Cluster")

# Plotting the data using matplotlib.pyplot
plt.figure(figsize=(25,5))
plt.bar(unique_industries_per_cluster["Cluster"], unique_industries_per_cluster["Industry"],)
plt.xlabel("Cluster")
plt.ylabel("Number of Unique Industries")
plt.title("Number of Unique Industries per Cluster")
plt.xticks(unique_industries_per_cluster["Cluster"].tolist())
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


#### Evaluate

In [ ]:
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(stock_prices_normal, kmeans.labels_))

print("Calinski-Harabasz Index: %0.3f"
      % metrics.calinski_harabasz_score(stock_prices_normal, kmeans.labels_))

print("Davies-Bouldin Index: %0.3f"
      % metrics.davies_bouldin_score(stock_prices_normal, kmeans.labels_))

#### Explore

##### Clusters

In [ ]:
plt.figure(figsize=(20,15))
plt.scatter(mySeries_transformed[:, 0], mySeries_transformed[:, 1], c=labels, s=300)
plt.show()

In [ ]:
for cluster in range(0, kmeans_cluster_count):
  plt.figure(figsize=(25,5))
  plt.title(f"Cluster {cluster + 1}")
  for index, clusterNo in enumerate(labels):
    if clusterNo == cluster:
      plt.plot(stock_prices_normal[index])
  plt.show()

##### Averaging Centre

In [ ]:
plot_count = math.ceil(math.sqrt(kmeans_cluster_count))

fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
row_i=0
column_j=0
for label in set(labels):
    cluster = []
    for i in range(len(labels)):
            if(labels[i]==label):
                axs[row_i, column_j].plot(stock_prices_normal[i],c="gray",alpha=0.4)
                cluster.append(stock_prices_normal[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(row_i*som_y+column_j))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0

plt.show()

#### DBA

In [ ]:
plot_count = math.ceil(math.sqrt(kmeans_dtw_cluster_count))

fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
row_i=0
column_j=0
for label in set(labels):
    cluster = []
    for i in range(len(labels)):
            if(labels[i]==label):
                axs[row_i, column_j].plot(stock_prices_normal[i],c="gray",alpha=0.4)
                cluster.append(stock_prices_normal[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(dtw_barycenter_averaging(np.vstack(cluster)),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(row_i*plot_count+column_j + 1))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0


plt.show()

#### Distributions

In [ ]:
cluster_c = [len(labels[labels==i]) for i in range(kmeans_cluster_count)]
cluster_n = [str(i + 1) for i in range(kmeans_cluster_count)]
plt.figure(figsize=(25,5))
plt.title("Cluster Distribution for PCA KMeans DTW")
plt.bar(cluster_n,cluster_c)
plt.show()

In [ ]:
fancy_names_for_labels = [f"Cluster {label}" for label in labels]
pd.DataFrame(zip(symbols,fancy_names_for_labels, [companies_profiles[d]["industry"].values[0] for d in symbols]),
             columns=["Series","Cluster", "Industry"]).sort_values(by="Cluster").set_index("Series")

# Comparison

# Conclusions